In [1]:
import datetime
import numpy as np
import matplotlib.pyplot as plt
from keras.layers import Input, Activation, Conv2D, Flatten, Dense, MaxPooling2D, Dropout
from keras.models import Model, load_model
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras.callbacks import TensorBoard
from keras.utils.vis_utils import plot_model

tb_hist = TensorBoard(log_dir='./graphFINAL', histogram_freq=0, write_graph=True, write_images=True)

# Load Dataset

import glob

imgs = glob.glob('./mrlEyes_2018_01/*/*.png')

from sklearn.model_selection import train_test_split

train_imgs, test_imgs = train_test_split(imgs, test_size=0.15)

len(train_imgs), len(test_imgs)

train_imgs, valid_imgs = train_test_split(train_imgs, test_size=0.18)

len(train_imgs), len(valid_imgs), len(test_imgs)

from collections import Counter
from PIL import Image

x_train = np.empty((len(train_imgs), 32, 32, 1))
y_train = np.empty(len(train_imgs))

x_valid = np.empty((len(valid_imgs), 32, 32, 1))
y_valid = np.empty(len(valid_imgs))

x_test = np.empty((len(test_imgs), 32, 32, 1))
y_test = np.empty(len(test_imgs))

for idx, train in enumerate(train_imgs):
    x_train[idx] = np.expand_dims(np.array(Image.open(train).resize((32, 32), Image.BICUBIC)), -1)
    y_train[idx] = int(train.split('/')[-1].split('_')[4])

for idx, valid in enumerate(valid_imgs):
    x_valid[idx] = np.expand_dims(np.array(Image.open(valid).resize((32, 32), Image.BICUBIC)), -1)
    y_valid[idx] = int(valid.split('/')[-1].split('_')[4])

for idx, test in enumerate(test_imgs):
    x_test[idx] = np.expand_dims(np.array(Image.open(test).resize((32, 32), Image.BICUBIC)), -1)
    y_test[idx] = int(test.split('/')[-1].split('_')[4])

x_train.shape, y_train.shape, x_valid.shape, y_valid.shape, x_test.shape, y_test.shape

# Preview

plt.subplot(2, 1, 1)
plt.title(str(int(y_train[0])))
plt.imshow(x_train[0].reshape((32, 32)), cmap='gray')
print("\n")
plt.subplot(2, 1, 2)
plt.title(str(int(y_valid[4])))
plt.imshow(x_valid[4].reshape((32, 32)), cmap='gray')

# Data Augmentation

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=10,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2
)

valid_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=10,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2
)

test_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=10,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2
)


#valid_datagen = ImageDataGenerator(rescale=1./255)

#test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow(
    x=x_train, y=y_train,
    batch_size=32,
    shuffle=True
)

valid_generator = valid_datagen.flow(
    x=x_valid, y=y_valid,
    batch_size=32,
    shuffle=True
)

test_generator = test_datagen.flow(
    x=x_test, y=y_test,
    batch_size=32,
    shuffle=True
)

# Build Model

inputs = Input(shape=(32, 32, 1))

# net = Conv2D(32, kernel_size=3, strides=1, padding='same', activation='relu')(inputs)
# net = MaxPooling2D(pool_size=2)(net)

# net = Conv2D(64, kernel_size=3, strides=1, padding='same', activation='relu')(net)
# net = MaxPooling2D(pool_size=2)(net)

# net = Conv2D(128, kernel_size=3, strides=1, padding='same', activation='relu')(net)
# net = MaxPooling2D(pool_size=2)(net)

# net = Flatten()(net)

# net = Dense(512)(net)
# net = Activation('relu')(net)
# net = Dense(1)(net)
# outputs = Activation('sigmoid')(net)

Using TensorFlow backend.
/usr/local/lib/python3.5/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.5/dist-packages/tensorflow/python/framework/dtypes.py:520: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.5/dist-packages/tensorflow/python/framework/dtypes.py:521: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.5/dist-packages/tensorflow/python/framework/dtypes.py:522: FutureWarn

In [2]:
net = Conv2D(32, kernel_size=3, strides=1, padding='same', activation='relu')(inputs)
net = MaxPooling2D(pool_size=2)(net)
#net = BatchNormalization(axis=chanDim)(net)
net = Dropout(0.25)(net)

net = Conv2D(64, kernel_size=3, strides=1, padding='same', activation='relu')(net)
net = MaxPooling2D(pool_size=2)(net)
#net = BatchNormalization(axis=chanDim)(net)
net= Dropout(0.25)(net)


net = Conv2D(128, kernel_size=3, strides=1, padding='same', activation='relu')(net)
net = MaxPooling2D(pool_size=2)(net)
#net = BatchNormalization(axis=chanDim)(net)
net = Dropout(0.25)(net)

net = Flatten()(net)

net = Dense(512)(net)
net = Activation('relu')(net)
#net = BatchNormalization()(net)
net = Dropout(0.5)(net)


#classifier
net = Dense(1)(net)
outputs = Activation('sigmoid')(net)

model = Model(inputs=inputs, outputs=outputs)

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

plot_model(model, to_file = './model_customize.png', show_shapes =True, show_layer_names=True)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 32, 32, 1)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 32, 32)        320       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 16, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 16, 16, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 8, 8, 64)          0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 8, 8, 64)          0         
__________

In [ ]:


# Train

start_time = datetime.datetime.now().strftime('%Y_%m_%d_%H_%M_%S')
epochs_num = 50
hist = model.fit_generator(
    train_generator, epochs= epochs_num , validation_data=valid_generator,
    callbacks=[
        ModelCheckpoint('models/%s.h5' % (start_time), monitor='val_loss', save_best_only=True, mode='max', verbose=1),
        ReduceLROnPlateau(monitor='valid_acc', factor=0.2, patience=10, verbose=1, mode='auto', min_lr=1e-05), tb_hist
    ]
)

model.save('models/train_mrl_%s.h5' %(epochs_num))



import matplotlib.pyplot as plt

plt.subplot(2, 1, 1)
plt.title(str(int(y_train[0])))
plt.imshow(x_train[0].reshape((32, 32)), cmap='gray')
plt.subplot(2, 1, 2)
plt.title(str(int(y_valid[4])))
plt.imshow(x_valid[4].reshape((32, 32)), cmap='gray')


fig, loss_ax = plt.subplots()

acc_ax = loss_ax.twinx()

loss_ax.plot(hist.history['loss'], 'y', label='train loss')
loss_ax.plot(hist.history['val_loss'], 'r', label='val loss')
loss_ax.set_ylim([0.0, 0.5])

acc_ax.plot(hist.history['acc'], 'b', label='train acc')
acc_ax.plot(hist.history['val_acc'], 'g', label='val acc')
acc_ax.set_ylim([0.8, 1.0])

loss_ax.set_xlabel('epoch')
loss_ax.set_ylabel('loss')
acc_ax.set_ylabel('accuray')

loss_ax.legend(loc='upper left')
acc_ax.legend(loc='lower left')

plt.show()


scores = model.evaluate_generator(test_generator, steps=10)
print("## evaluation ##")
print(scores)
print("##  accuracy  ##")
print("%s: %.2f%%" %(model.metrics_names[1], scores[1]*100))

scores = model.evaluate_generator(test_generator)
print("## evaluation ##")
print(scores)
print("##  accuracy  ##")
print("%s: %.2f%%" %(model.metrics_names[1], scores[1]*100))

Epoch 1/50
1850/1850 [==============================] - 28s 15ms/step - loss: 0.4246 - acc: 0.7952 - val_loss: 0.2325 - val_acc: 0.9142
Epoch 2/50
  19/1850 [..............................] - ETA: 16s - loss: 0.2715 - acc: 0.8931

/usr/local/lib/python3.5/dist-packages/keras/callbacks.py:432: RuntimeWarning: Can save best model only with valid_acc available, skipping.
  'skipping.' % (self.monitor), RuntimeWarning)
/usr/local/lib/python3.5/dist-packages/keras/callbacks.py:1043: RuntimeWarning: Reduce LR on plateau conditioned on metric `valid_acc` which is not available. Available metrics are: lr,loss,acc,val_acc,val_loss
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


1850/1850 [==============================] - 26s 14ms/step - loss: 0.2145 - acc: 0.9149 - val_loss: 0.1336 - val_acc: 0.9506
Epoch 3/50
1850/1850 [==============================] - 26s 14ms/step - loss: 0.1616 - acc: 0.9375 - val_loss: 0.0986 - val_acc: 0.9647
Epoch 4/50
1850/1850 [==============================] - 26s 14ms/step - loss: 0.1324 - acc: 0.9495 - val_loss: 0.0915 - val_acc: 0.9679
Epoch 5/50
1850/1850 [==============================] - 26s 14ms/step - loss: 0.1221 - acc: 0.9547 - val_loss: 0.0955 - val_acc: 0.9638
Epoch 6/50
1850/1850 [==============================] - 26s 14ms/step - loss: 0.1156 - acc: 0.9572 - val_loss: 0.0695 - val_acc: 0.9740
Epoch 7/50
1850/1850 [==============================] - 26s 14ms/step - loss: 0.1094 - acc: 0.9596 - val_loss: 0.0735 - val_acc: 0.9723
Epoch 8/50
1850/1850 [==============================] - 26s 14ms/step - loss: 0.1001 - acc: 0.9629 - val_loss: 0.0605 - val_acc: 0.9782
Epoch 9/50
1850/1850 [==============================] - 27s

In [69]:
#start_time = datetime.datetime.now().strftime('%Y_%m_%d_%H_%M_%S')
#
#model.fit_generator(
#    train_generator, epochs=50, validation_data=valid_generator,
#    callbacks=[
#        ModelCheckpoint('models/%s.h5' % (start_time), monitor='valid_acc', save_best_only=True, mode='max', verbose=1),
#        ReduceLROnPlateau(monitor='valid_acc', factor=0.2, patience=10, verbose=1, mode='auto', min_lr=1e-05)
#    ]
#)

In [70]:
#model.save('models/train_mrl_0.h5')

In [71]:
#model.save(‘train_mrl_0.hdf5’)

In [72]:
#history['loss'], history['val_loss']

# Confusion Matrix

In [ ]:
#from sklearn.metrics import accuracy_score, confusion_matrix
#import seaborn as sns

#model = load_model('models/%s.h5' % (start_time))

#y_pred = model.predict(x_val/255.)
#y_pred_logical = (y_pred > 0.5).astype(np.int)

#print ('test acc: %s' % accuracy_score(y_val, y_pred_logical))
#cm = confusion_matrix(y_val, y_pred_logical)
#sns.heatmap(cm, annot=True)

# Distribution of Prediction

In [ ]:
#ax = sns.distplot(y_pred, kde=False)

---

# MobileNet V2

In [ ]:
#from keras.applications import MobileNetV2

In [ ]:
#model = MobileNetV2(input_shape=(32, 32, 1), include_top=True, weights=None, classes=2)

In [ ]:
#model.summary()